# import packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("src/")
from src import *

2024-11-21 09:48:16.923253: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 09:48:16.941879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732153696.963376  147526 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732153696.969993  147526 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 09:48:16.993200: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# data preprocess

In [ ]:
# data_pp = DataProcessor(exp_path=f"./data/phos_MS_merged/{ct}/{ct}.feather", 
#                         ksr_path=f"data/KSR/prior_{ct}.tsv", 
#                         output_path=f"./data/preprocessed_data/{ct}", 
#                         with_centering=False, quantile_range=(1, 99), unit_variance=False)
# data, prior = data_pp.process_all()

In [ ]:
data.shape, prior.shape

# run

In [3]:
ct = 'LA'

data = pd.read_parquet(f"data/preprocessed_data/{ct}/data.parquet")
prior = pd.read_parquet(f"data/preprocessed_data/{ct}/prior.parquet")

data.shape, prior.shape

((274, 10287), (186, 10287))

In [ ]:
dp_list = [0, 0.05, 0.1, 0.3, 0.5]

dp1_list = [0.3, 0.5]

len(dp_list)

In [5]:
data
data = np.array(data).flatten()
percentile_80 = np.percentile(data, 80)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

for idx, (index, row) in enumerate(data.iterrows()):
    if idx < 100  and idx > 90:  # 限制只显示前 10 个
    # print(f"index = {index}")
    # print(f"row = {row}")
        plt.figure(figsize=(10, 6))
        plt.hist(row.values, bins=30, alpha=0.6, color='g', edgecolor='black', density=True)
        plt.title(f'Data Distribution (Histogram) - Index {index}', fontsize=16)
        plt.xlabel('Value', fontsize=12)
        plt.ylabel('Density', fontsize=12)
        plt.grid(True)

        plt.figure(figsize=(10, 6))
        sns.kdeplot(row.values, shade=True, color='blue')
        plt.title(f'Data Distribution (Density Plot) - Index {index}', fontsize=16)
        plt.xlabel('Value', fontsize=12)
        plt.ylabel('Density', fontsize=12)
        plt.grid(True)

plt.show()




In [ ]:
count = (data < 1.2).sum().sum()  
total_elements = data.size

percentage = (count / total_elements) * 100
percentage

In [ ]:
for dp1 in dp1_list:
    for dp2 in dp_list:
        print(f"dp1: {dp1}, dp2: {dp2}\n")

        fit_model(data, prior, output_path=f"./result_{ct}_changeloss_threshold_1.2/dp1_{dp1}_dp2_{dp2}/", 
                  data_val_size=0.3, batch_size=32, fraction_gs=0.2, 
                  num_epochs=200, cvs=5, num_epochs_refit=50, refit_iters=10, refit_resample=True, 
                  weight_decays=(-10, -1, 4), lr=1e-4, 
                  scheduler_class=torch.optim.lr_scheduler.CosineAnnealingLR, scheduler_kwargs={'T_max': 10}, 
                  optimizer_class=torch.optim.Adam, optimizerkw={}, optimizer_paramskw={},
                  dropout_rate1=dp1, dropout_rate2=dp2, dropout_rate3=dp2, activation=ReLU0(),
                  eps=torch.finfo(torch.float).eps, eps_factor=10, fill_zeroed=True, device='cuda:1')

In [ ]:
dp1 = 0.05
dp2 = 0
print(f"dp1: {dp1}, dp2: {dp2}\n")
fit_model(data, prior, output_path=f"./result_{ct}_percentile_test/dp1_{dp1}_dp2_{dp2}/", 
          data_val_size=0.3, batch_size=32, fraction_gs=0.2, 
          num_epochs=200, cvs=5, num_epochs_refit=50, refit_iters=10, refit_resample=True, 
          weight_decays=(-10, -1, 4), lr=1e-4, 
          scheduler_class=torch.optim.lr_scheduler.CosineAnnealingLR, scheduler_kwargs={'T_max': 10}, 
          optimizer_class=torch.optim.Adam, optimizerkw={}, optimizer_paramskw={},
          dropout_rate1=dp1, dropout_rate2=dp2, dropout_rate3=dp2, activation=ReLU0(),
          eps=torch.finfo(torch.float).eps, eps_factor=10, fill_zeroed=True, device='cuda:0',alpha = 0.8,percentile = 80)

dp1: 0.05, dp2: 0

percentile = 80,threshold = 1.1380871362160427
Total number of KSRs: 93061, number of KSRs used to train: 74530, number of KSRs used to test: 18531

cv: 0, weight_decay: 1e-10
Epoch [200/200], Train Loss: 0.0100, Val Loss: 0.0106, Val R2: 0.9019

cv: 0, weight_decay: 1e-10, refit: 0
Epoch [50/50], Train Loss: 0.0358, Val Loss: 0.0342, Val R2: 0.6896

cv: 0, weight_decay: 1e-10, refit: 1
Epoch [50/50], Train Loss: 0.0391, Val Loss: 0.0397, Val R2: 0.6256

cv: 0, weight_decay: 1e-10, refit: 2
Epoch [50/50], Train Loss: 0.0342, Val Loss: 0.0347, Val R2: 0.6778

cv: 0, weight_decay: 1e-10, refit: 3
Epoch [50/50], Train Loss: 0.0295, Val Loss: 0.0288, Val R2: 0.7283

cv: 0, weight_decay: 1e-10, refit: 4
Epoch [50/50], Train Loss: 0.0286, Val Loss: 0.0275, Val R2: 0.7355

cv: 0, weight_decay: 1e-10, refit: 5
Epoch [50/50], Train Loss: 0.0272, Val Loss: 0.0283, Val R2: 0.7219

cv: 0, weight_decay: 1e-10, refit: 6
Epoch [50/50], Train Loss: 0.0269, Val Loss: 0.0274, Val R2: 